In [40]:
from time import sleep

from pyspark import StorageLevel
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local").getOrCreate()
sc = spark.sparkContext

### Как создать RDD из массива

In [41]:
numbers = range(1, 1000000)
numbers_parent_rdd = sc.parallelize(numbers, 4)

In [42]:
print(numbers_parent_rdd.toDebugString().decode('utf-8'))
num2 = numbers_parent_rdd.take(10)
print(num2)
print("Количество разделов = ",numbers_parent_rdd.getNumPartitions())

### Как создать RDD из файла

In [43]:
# sc.textFile возвращает RDD - распределенную коллекцию Java-объектов
stocks_rdd_v2 = sc.textFile("data/stocks/aapl.csv"). \
    map(lambda row: row.split(",")). \
    filter(lambda tokens: float(tokens[2]) > 15)

# пока файл еще не читался
# даже не было проверки его существования
print(stocks_rdd_v2.toDebugString().decode('utf-8'))

In [44]:
stocks_rdd_v2.take(10)

In [45]:
# spark.read возвращает DataFrame - табличное представление данных 
stocks_df = spark.read.csv("data/stocks"). \
    withColumnRenamed("_c0", "company"). \
    withColumnRenamed("_c1", "date"). \
    withColumnRenamed("_c2", "price")

# из DataFrame можно извлечь хранимые данные в виде RDD
stocks_rdd_v3 = stocks_df.rdd

prices_rdd = stocks_rdd_v3.map(lambda row: row.price)
print(prices_rdd.toDebugString().decode('utf-8'))

# до сих пор было только планирование работы
# читать и обрабатывать данные будем только сейчас
prices_rdd.take(10)

In [46]:
# из RDD[Row] сделать DataFrame
stocks_df_v2 = spark.createDataFrame(stocks_rdd_v3)
print(stocks_df_v2.columns)
stocks_df_v2.take(10)

### Зачем может понадобиться RDD
- алгоритм обработки, который сложно выразить на SQL
- особые оптимизации

In [47]:
# ленивые преобразования: из одного RDD сделать другой RDD
# например, map, filter, flatMap, distinct
company_names_rdd = stocks_rdd_v3 \
    .map(lambda row: row.company) \
    .distinct()
# действие take запускает работу
# нужно вернуть реальные данные, лениво выполнить не получится
company_names_rdd.take(10)

In [48]:
# действие count запускает работу
# нужно вернуть реальное значение, лениво выполнить не получится
total_entries = stocks_rdd_v3.count()  
total_entries

In [49]:
aapl_stocks_rdd = stocks_rdd_v3 \
    .filter(lambda row: row.company == "AAPL") \
    .map(lambda row: float(row.price))

# действия min и max
max_aapl = aapl_stocks_rdd.max()
min_aapl = aapl_stocks_rdd.min()
print(max_aapl)
print(min_aapl)

In [50]:
# действие reduce 
sum_prices = aapl_stocks_rdd \
    .reduce(lambda x, y: x + y)  # здесь может быть любая функция / код, возвращающий значение
#  reduce (1,2,3,4)
#  1+2 = 3 
#        3+3 = 6 
#                6+4 = 10
print(sum_prices)

In [51]:
# группировка возвращает RDD с группаими в качестве элементов
grouped_stocks_rdd = stocks_rdd_v3 \
    .groupBy(lambda row: row.company)  # в одну группу попадут все элементы с одинаковым значением этой функци
# реализация группировки - всегда две фазы типа Map и Reduce + Shuffle между ними
grouped_stocks_rdd.map(lambda value: (value[0], list(value[1])[:5])).take(10)

In [52]:
print("Оригинальное колличество партиций:", stocks_rdd_v3.getNumPartitions())
# изменение количества разделов: полная перегруппировка 
# Shuffle
repartitioned_stocks_rdd = stocks_rdd_v3.repartition(4)
print("Произошло увеличение партиций (1 -> 4):", repartitioned_stocks_rdd.getNumPartitions())

# изменение количества разделов: попарное слияние разделов, пока их не станет нужное количество 
# без Shuffle
repartitioned_stocks_rdd1 = stocks_rdd_v3.coalesce(2)
print("Увеличения партиций не произошло (1 -> 1):", repartitioned_stocks_rdd1.getNumPartitions())

repartitioned_stocks_rdd = repartitioned_stocks_rdd.coalesce(2)
print("Произошло уменьшение партиций (4 -> 2):", repartitioned_stocks_rdd.getNumPartitions())

# RDD
#  part1 => |||||| 20           
#  part2 => |||||||||||||| 40   
#  part3 => ||||| 10            
#  part4 => ||||| 10       
# максимальный параллелизм = количество исполнителей * количество ядер у исполнителя

# rdd.repartition(2) - происходит shuffle, равномерное перераспределение
#  part1 => ||||| 40
#  part2 => ||||| 40

# rdd.repartition(8) - происходит shuffle, равномерное перераспределение
#  part1 => ||||| 10
#  part2 => ||||| 10
#  part3 => ||||| 10
#  part4 => ||||| 10
#  part5 => ||||| 10
#  part6 => ||||| 10
#  part7 => ||||| 10
#  part8 => ||||| 10

# rdd.coalesce(10) - ничего не меняется, coalesce умеет только склеивать и не умеет разделять

# rdd.coalesce(2) - склеиваются соседние разделы
# part1 => |||||| 20 + |||||||||||||| 40 => 60
# part2 => ||||| 10 +   ||||| 10 => 20

# rdd.coalesce(1) - склеиваются соседние разделы

# rdd.repartition(col("company")) - неравномерное распределение по разделам


### Как сохранить RDD на диск

In [53]:
r = [1, 2, 3, 4, 5, 6, 7, 8]
ints = sc.parallelize(r, 4).coalesce(2)
print(ints.getNumPartitions())

In [54]:
! rm -rf data/output/ints

In [55]:
ints.saveAsTextFile("data/output/ints")
print("Saved")

In [56]:
! rm -rf data/output/ints2

In [57]:
ints = sc.parallelize(r).coalesce(1)
ints.coalesce(2) \
    .saveAsTextFile("data/output/ints2")

In [58]:
cachedInts = sc.textFile("data/output/ints2") \
    .map(lambda x: int(x)) \
    .persist(StorageLevel.DISK_ONLY_2) 

In [59]:
print(cachedInts.count())
print(cachedInts.take(1))

In [60]:
cachedInts.unpersist()
print(cachedInts.count())

In [61]:
unpersisted = cachedInts.map(lambda x: x + 1).take(5)
print(unpersisted)

In [62]:
reduced = cachedInts.reduce(lambda x, y: x + y)
print(reduced)

In [63]:
doubles = cachedInts.map(lambda x: x * 2)

even = cachedInts.filter(lambda x: x % 2 == 0)

print(even.collect())

even.setName("Even numbers")
print("Name is " + even.name() + " id is " + str(even.id()))

plan = even.toDebugString().decode("utf-8")

print(plan)
print(doubles.take(5))

### Как группировать и соединять RDD

In [64]:
data = [("Ivan", 240), ("Petr", 39), ("Elena", 290), ("Elena", 300)]
codeRows = sc.parallelize(data)
codeRows.collect()

In [65]:
reduced = codeRows.reduceByKey(lambda x, y: x + y)
print(reduced.collect())

In [66]:
deduplicated = codeRows.reduceByKey(lambda x, y: x if (x > y) else y)
print(deduplicated.collect())

In [67]:
folded = codeRows.foldByKey(1000, lambda x, y: x + y)

# part1 (k1:2, k2:2, k3:2, k1:2) => shufle => reduce (k1:2, k1:2, k1:2) => k1:6
# part2 (k2:2, k2:2, k3:2, k1:2) => shufle => (k2:2, k2:2, k2:2) => k2:6, (k3:2, k3:2) => k3:4

folded.collect()

In [68]:
aggregated = codeRows.aggregateByKey(1000, lambda x, y: x + y, lambda x, y: x + y)
aggregated.collect()

#  part1 (k1:2, k2:2, k3:2, k1:2) => (k1:4, k2:2, k3:2) =>  shuffle => (k1:4, k1:2) => k1:6
#  part2 (k2:2, k2:2, k3:2, k1:2) => (k1:2, k2:4, k3:2) => shuffle => (k2:4, k2:2) => k2:6, (k3:2, k3:2) => k3:4

In [69]:
grouped = codeRows.groupByKey()
print(grouped.collect())

plan = grouped.toDebugString().decode("utf-8")
print(plan)

### Соединение

In [70]:
profileData = [("Ivan", "Java"), ("Elena", "Scala"), ("Petr", "Scala")]
programmerProfiles = sc.parallelize(profileData)
programmerProfiles.collect()

In [71]:
joined = programmerProfiles.join(codeRows)
print(joined.toDebugString().decode("utf-8"))
joined.collect()

In [72]:
data = [("Ivan", 240), ("Petr", 39), ("Elena", 290), ("Elena", 300)]
codeRows = sc.parallelize(data)
codeRows = programmerProfiles.cogroup(codeRows)
codeRows.map(lambda row: (row[0], [list(k) for k in row[1]])).take(5)

In [73]:
programmerProfiles.cogroup(codeRows) \
    .sortByKey(False) \
    .collect()

In [74]:
print("== CountByKey")
print(joined.countByKey())

In [75]:
print("== Keys")
codeRows.keys().collect()

In [76]:
print("== Value")
codeRows.values().map(lambda x : [list(k) for k in x]).collect()